In [1]:
from qiskit import execute, transpile
from qiskit.providers.aer import AerSimulator
from qiskit.circuit.library import QuantumVolume
from dask.distributed import Client

ImportError: cannot import name 'execute' from 'qiskit' (/opt/miniconda3/envs/qiskit/lib/python3.11/site-packages/qiskit/__init__.py)

In [2]:
from dask_jobqueue import LSFCluster
from dask.distributed import Client
from qiskit.providers.aer import AerSimulator

# Initialise your HPC cluster
my_hpc_cluster = LSFCluster(cores=4, memory="30GB")

# Initialise DASK client with the HPC cluster
my_dask_client = Client(my_hpc_cluster)

# Initialise your Aer simulator
backend = AerSimulator()

# Set the client as a custom executor for your backend
backend.set_options(executor=my_dask_client, max_job_size=1)

# Run your circuits
result = backend.run(circ_list).result()

ModuleNotFoundError: No module named 'dask_jobqueue'